In [1]:
from utils import * 
from algorithms import *
from agents import *

In [2]:
numRounds, numPeriods, numSteps = 1, 5000, 10
numBuyers, numSellers, numTokens = 4, 4, 4
gameTypes, seed = '1111', 42
disclosure = []
depth = 0
gameData = [gameTypes, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps, seed]
verbose = 1
num_states = numStates = 2
buyers = [Reinforcer(gameData, disclosure, index=0, buyer=1, reinforcer=1, numStates = numStates, algo='SAC1', algoArgs = [num_states, verbose]),
          ZeroIntelligence(gameData, disclosure, index=0, buyer=1, reinforcer=0),
          ZeroIntelligence(gameData, disclosure, index=0, buyer=1, reinforcer=0),
          ZeroIntelligence(gameData, disclosure, index=0, buyer=1, reinforcer=0)]
sellers = [ZeroIntelligence(gameData, disclosure, index=0, buyer=0, reinforcer=0),
           ZeroIntelligence(gameData, disclosure, index=0, buyer=0, reinforcer=0),
           ZeroIntelligence(gameData, disclosure, index=0, buyer=0, reinforcer=0),
           ZeroIntelligence(gameData, disclosure, index=0, buyer=0, reinforcer=0)]
log = Log(gameData, disclosure)
rnd = 0

In [3]:
print_interval = 10
for rnd in range(numRounds):
    startRounds(gameData, log, buyers, sellers, rnd)
    #print(log.roundData.buyerValues.item())
    score = 0.0
    for period in range(numPeriods):
        startPeriods(buyers, sellers)
        for step in range(numSteps):
            # start
            startSteps(buyers, sellers)

            # set state
            buyers[0].state = [buyers[0].periodSteps, buyers[0].periodTrades]

            # trade
            bids, asks = collectOffers(buyers, sellers)
            currentAsk, currentAskIdx, currentBid, currentBidIdx = bestOffers(bids, asks)
            price, buy, sell = trade(buyers, sellers, currentAsk, currentAskIdx, currentBid, currentBidIdx)
            bprofit, sprofit = 0, 0
            if price > 0:
                buyers[currentBidIdx].transact(price)
                sellers[currentAskIdx].transact(price)
                bprofit = buyers[currentBidIdx].stepProfits
                sprofit = sellers[currentAskIdx].stepProfits
            if currentBidIdx == 0:
                score += bprofit
                
            # log and end step
            log.addStep([rnd, period, step, bids, asks, currentBid, currentBidIdx, currentAsk, currentAskIdx, buy, sell, price, price>0, bprofit, sprofit])
            observe(buyers, sellers, log.disclose())
            endSteps(buyers, sellers) # updates period counters

            # new state and done
            buyers[0].newState = [step+1, buyers[0].periodTrades]
            if buyers[0].periodSteps == numSteps - 1:
                buyers[0].done = True
            else:
                buyers[0].done = False

            # load data and train
            agentsObserve(buyers, sellers)
            agentsTrain(buyers, sellers)

        endPeriods(buyers, sellers)
        if period%print_interval==0:
            print(score/print_interval)
            score = 0.0
    endRounds(buyers, sellers)

8.682149826254596
35.626562636567996
47.366850948379735
43.22035319209231
33.084091488255
36.573579586030725
46.17033049279094
43.84089880923189
50.21793387362901
47.32362990124073
46.21773741578441
39.911838102849075
41.638370818146704
45.28886249602464
43.149127663124574
30.209009799290435
48.62754926219385
44.26964438594245
39.56972154088033
50.518982507410726
30.217152840905634
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


RuntimeError: Function 'TanhBackward0' returned nan values in its 0th output.

In [ ]:
log.graphLearning(rolling_window = 100, trackBuyersIdx=[0], trackSellersIdx=[0])

In [ ]:
log.graphSales(0, period-1, trackBuyersIdx=[0], trackSellersIdx=[0])

In [ ]:
log.roundData.buyerValues.item()

In [ ]:
log.stepData.tail(15)

In [ ]:
log.graphOffers(0, period-1, trackBuyersIdx=[0], trackSellersIdx=[0])

In [ ]:
log.graphTraining(maxNumPeriods=period-1, skip = int((period-1)/50), trackBuyersIdx = [0])